In [1]:
# https://www.pinecone.io/learn/data-augmentation/ augmentedSBERT
# https://github.com/snunlp/KR-SBERT 한국어 SBERT
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [2]:
# sentences = ['잠이 옵니다', '졸음이 옵니다', '기차가 옵니다']
# vectors = model.encode(sentences) # encode sentences into vectors
# print(vectors.shape)
# similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
# print(similarities)

문장을 벡터화해서 문장 간 유사도를 구하는 모델.<br>
그러면 이걸 가지고 multi-label 분류를 할 수도 있지 않을까?<br>
벡터화를 일단 시키고 그걸 인공신경망에 넣어서 11개의 태스크로 분류하기(multi-label)

In [28]:
# import
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras import backend as K
from sentence_transformers import SentenceTransformer, util, models, losses
import tensorflow as tf
import tensorflow_addons as tfa
import wandb
import torch
from tensorflow.keras.metrics import BinaryAccuracy, AUC
from tensorflow_addons.metrics import F1Score
from keras.regularizers import l1, l2
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import math

In [4]:
wandb.init(project='cp2_s_bert')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nemyrmain. Use `wandb login --relogin` to force relogin


In [5]:
filepath = 'reviews_label/review_w_only_labels (3).csv'
df = pd.read_csv(filepath)
columns = df.columns[1:].tolist()
columns.remove('review_tags')
columns.remove('target')
df = df[columns]
df

,review,가성비,귀여운,넓은,단체,만족,모던,분위기,비주얼,아늑,위생,응대,이색음식,이색테마,클래식,혼자
0,고기도 구워주시고 친절도 하시고 맛도 최고입니다.,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,가게 인기가 좋아서 그런지 가게가 바빴는데 직원분들 저너무 친절하시구 초벌 도와주신...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,공간도 깔끔하고 맛있습니다.,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,웨이팅은 했지만 깔끔하고 송년모임 하기 좋은것같아요 다음에 올때는 6시전에 좀 일찍...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,직원분들이 친절하고 좋아용.리뷰만 보고 방문했는데 이렇게 분위기 좋은 고깃집은 첨이...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,"치즈가 쭉쭉 늘어날 정도로 많이 들어있고, 튀김은 엄청 바삭하고, 김밥은 너무 맛있...",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2610,양많고 맛나요.치즈불맛덕에 더 특별한 맛.테이블은 조금 좁아요,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2611,치즈 폭포 진짜 갬동 그자체,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2612,떡볶이중독자라 인스타에서 우연히 보고 먹으러왔어요 매운떡볶이를 좋아하지만 치즈가 가...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
targets = df.columns.tolist()[1:]
targets

['가성비',
 '귀여운',
 '넓은',
 '단체',
 '만족',
 '모던',
 '분위기',
 '비주얼',
 '아늑',
 '위생',
 '응대',
 '이색음식',
 '이색테마',
 '클래식',
 '혼자']

In [7]:
df_melt = pd.melt(df.reset_index(), id_vars=['index', 'review'])
df_melt = df_melt[df_melt['index'] < 50]
df = df.loc[50:]
df.reset_index(drop=True)
df

,review,가성비,귀여운,넓은,단체,만족,모던,분위기,비주얼,아늑,위생,응대,이색음식,이색테마,클래식,혼자
50,고등어 덮밥 이 녀석 정말 혁명.제 원픽 메뉴 너무 맛있어요.,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
51,고등어덮밥과 크림버섯누룽지를 꼭 드세요.,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
52,맛있어요식사류부터 안주류까지.음료도 다양하게 있어서 골라먹는 재미가있어요 사장님도 ...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
53,고등어 덮밥 너무 맛있어요,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54,고등어덮밥 맛있다고 친구가 꼬셔서 왔는데 정말 맛있네요.다음에 왔을때도 고등어덮밥은...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,"치즈가 쭉쭉 늘어날 정도로 많이 들어있고, 튀김은 엄청 바삭하고, 김밥은 너무 맛있...",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2610,양많고 맛나요.치즈불맛덕에 더 특별한 맛.테이블은 조금 좁아요,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2611,치즈 폭포 진짜 갬동 그자체,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2612,떡볶이중독자라 인스타에서 우연히 보고 먹으러왔어요 매운떡볶이를 좋아하지만 치즈가 가...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


SBERT 모델 train

In [49]:
s_bert_path = 'C:\AI_bootcamp_python\CP2/augSBERT_output'
word_embedding_model = models.Transformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=torch.nn.Tanh())

s_bert = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
# s_bert = SentenceTransformer(s_bert_path) # 학습 완료되면 이걸로 쓰기

In [50]:
num_epochs = 1
batch_size = 16

df_gold = df_melt[df_melt['index'] < 40].reset_index(drop=True) # 600개
df_eval = df_melt[df_melt['index'] >= 40].reset_index(drop=True) # 150개
gold_samples = []
dev_samples = []

for i in range(len(df_gold)):
    gold_samples.append(InputExample(texts=[df_gold.loc[i, 'review'], df_gold.loc[i, 'variable']], label=float(df_gold.loc[i, 'value'])))
for i in range(len(df_eval)):
    dev_samples.append(InputExample(texts=[df_eval.loc[i, 'review'], df_eval.loc[i, 'variable']], label=float(df_eval.loc[i, 'value'])))
train_dataloader = DataLoader(gold_samples, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(s_bert)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

# configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data

In [14]:
s_bert.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=s_bert_path
          )

Epoch: 100%|██████████| 1/1 [06:47<00:00, 407.79s/it]


문장 벡터화

In [15]:
X = df['review']
index = X.index
vectors = s_bert.encode(X.tolist())
vectors = pd.DataFrame(vectors)
vectors.index = index
vectors

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
50,0.061613,0.671017,-0.123301,-0.172436,0.134355,-0.542885,0.357799,0.252028,0.144963,0.094003,...,0.125844,0.196737,0.380382,0.259089,-0.008556,-0.385432,0.281497,-0.130010,0.381241,0.100413
51,-0.491869,0.341426,0.074147,0.122229,0.134118,-0.040123,0.167906,0.368079,-0.021788,0.566212,...,-0.035453,-0.313102,0.137247,-0.259951,0.197427,0.125005,-0.015631,-0.188259,0.458555,-0.156670
52,0.497886,0.774046,-0.098191,-0.484945,0.437741,-0.190475,-0.096258,-0.318769,-0.121178,0.573231,...,-0.049909,0.180891,-0.027598,0.254474,-0.306141,0.067562,0.325701,0.074086,0.345209,-0.186181
53,-0.211052,0.371984,-0.249619,-0.133433,0.215120,-0.339019,0.303869,0.474892,0.067742,0.412196,...,0.051815,-0.096628,0.495857,0.084821,0.177804,0.000037,0.133102,-0.056343,0.640249,-0.090148
54,-0.166933,0.636445,-0.420219,-0.266245,0.291507,-0.098149,0.509128,0.165812,0.174676,0.547795,...,0.301169,-0.037853,0.434125,-0.031750,0.038883,-0.223385,0.166889,0.095086,0.205761,-0.092506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,0.198299,0.395563,-0.062328,-0.013753,-0.070300,0.246679,0.195537,0.139859,0.022426,0.514346,...,-0.429742,0.268365,0.089940,0.127399,-0.245467,-0.031783,0.088107,0.205922,-0.052428,-0.076730
2610,0.495205,0.379784,0.513869,-0.475445,-0.167677,-0.060590,-0.083707,-0.281067,-0.310444,0.113320,...,-0.297664,0.590916,-0.259314,0.166560,-0.199633,-0.002721,-0.098279,0.241964,0.506430,-0.065359
2611,0.481121,0.862026,-0.110215,-0.318249,0.113683,0.017143,-0.281762,0.260843,-0.502561,0.396941,...,-0.053394,0.214685,-0.391691,-0.016629,-0.028282,-0.688978,-0.164564,0.390099,-0.261495,0.008144
2612,0.317057,0.616222,-0.156895,0.032037,0.253174,-0.071699,0.494969,-0.109560,-0.003500,0.628571,...,-0.023769,0.199057,0.130181,0.003456,-0.426705,-0.285568,0.373662,-0.239339,0.025630,-0.434593


In [17]:
y = df[targets]
y.sum()

가성비      347
귀여운       84
넓은       126
단체        38
만족       242
모던       160
분위기      720
비주얼       83
아늑       307
위생       265
응대      1000
이색음식     236
이색테마     160
클래식      116
혼자       103
dtype: int64

Sequential 모델 train

In [18]:
_vectors = np.array(vectors)
_y = np.array(y)
columns = y.columns

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=42, stratify=y)
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_test, y_test = iterative_train_test_split(_vectors, _y, test_size = 0.4)    # pandas.DataFrame이 아니라 np.array로 넣어야 함

In [20]:
df_y_train = pd.DataFrame(y_train)
df_y_train.columns = columns

In [21]:
class_weights = df_y_train.sum() / (len(df_y_train) - df_y_train.sum())  # 1인 개수 / 0인 개수
class_weights = 1 / class_weights   # (체크) 이거인가?
class_weights_list = class_weights.tolist()
class_weights = class_weights.to_dict()
i = 0
keys = list(class_weights.keys())
for k in keys:
    class_weights[i] = class_weights.pop(k)
    i += 1
class_weights

{0: 6.3798076923076925,
 1: 29.7,
 2: 19.19736842105263,
 3: 65.73913043478261,
 4: 9.586206896551724,
 5: 14.989583333333334,
 6: 2.553240740740741,
 7: 29.7,
 8: 7.342391304347826,
 9: 8.654088050314465,
 10: 1.5583333333333333,
 11: 9.809859154929576,
 12: 14.989583333333334,
 13: 21.246376811594203,
 14: 23.758064516129032}

In [47]:
from keras.regularizers import l1, l2, l1_l2

_accuracy = BinaryAccuracy()
_f1_score = F1Score(num_classes=15)
_auc = AUC(curve='PR', multi_label=True, num_labels=15)

# parameters
epochs = 150
batch_size = 64
wandb.config = {
  "epochs": epochs,
  "batch_size": batch_size
}

# tf.random.set_seed(42)
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l1_l2(0.0001, 0.0001)),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l1_l2(0.0001, 0.0001)),
    Dropout(0.2),
    Dense(15, activation='sigmoid')
])
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[_f1_score, _auc])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, class_weight=class_weights)

loss, f1_score, pr_auc = model.evaluate(X_test, y_test, batch_size=32)
print('Loss: ', loss)
print('f1_score: ', f1_score)
print('pr_auc: ', pr_auc)
f1_score_df = pd.DataFrame()
f1_score_df.index = targets
f1_score_df['f1_score'] = f1_score
f1_score_df

Epoch 1/150
24/24 [==============================] - 3s 9ms/step - loss: 5.0664 - f1_score: 0.0664 - auc_22: 0.1115
Epoch 2/150
24/24 [==============================] - 0s 7ms/step - loss: 3.8455 - f1_score: 0.1195 - auc_22: 0.1452
Epoch 3/150
24/24 [==============================] - 0s 7ms/step - loss: 3.5892 - f1_score: 0.1627 - auc_22: 0.1863
Epoch 4/150
24/24 [==============================] - 0s 6ms/step - loss: 3.4743 - f1_score: 0.1881 - auc_22: 0.2127
Epoch 5/150
24/24 [==============================] - 0s 7ms/step - loss: 3.3109 - f1_score: 0.2326 - auc_22: 0.2509
Epoch 6/150
24/24 [==============================] - 0s 6ms/step - loss: 3.2243 - f1_score: 0.2371 - auc_22: 0.2750
Epoch 7/150
24/24 [==============================] - 0s 6ms/step - loss: 3.1300 - f1_score: 0.2632 - auc_22: 0.2965
Epoch 8/150
24/24 [==============================] - 0s 7ms/step - loss: 3.0208 - f1_score: 0.2952 - auc_22: 0.3359
Epoch 9/150
24/24 [==============================] - 0s 7ms/step - loss:

,f1_score
가성비,0.649573
귀여운,0.600000
넓은,0.487805
단체,0.173913
만족,0.437500
모던,0.242424
분위기,0.612420
비주얼,0.217391
아늑,0.344444
위생,0.496894


In [ ]:
targets

['가성비',
 '귀여운',
 '넓은',
 '단체',
 '만족',
 '모던',
 '분위기',
 '비주얼',
 '아늑',
 '위생',
 '응대',
 '이색음식',
 '이색테마',
 '클래식',
 '혼자']

In [30]:
input = '''가성비 안 좋고 비싼 곳'''
def output(input, model):
    vector = s_bert.encode(input)
    vector = pd.DataFrame(vector).T
    return model.predict(vector)[0].tolist()
def labeling(input):
    return [1 if i > 0.5 else 0 for i in input]
temp = output(input, model)
labeled_temp = labeling(temp)
print([targets[i] for i in range(len(labeled_temp)) if labeled_temp[i]==1])
df_result = pd.DataFrame()
df_result.index = targets
df_result['점수'] = temp
df_result

1/1 [==============================] - 0s 143ms/step
['가성비']


,점수
가성비,0.988331
귀여운,0.000006
넓은,0.007227
단체,0.006767
만족,0.001701
모던,0.137796
분위기,0.000504
비주얼,0.000016
아늑,0.000136
위생,0.035506


In [ ]:
# save the model
# model.save('s_bert_model.h5', save_format='h5')
# new_model = keras.models.load_model('s_bert_model.h5')

In [ ]:
# save only model weights(not optimizer, architecture...)
# model.save_weights('s_bert_model_parameters.h5')


In [ ]:
# new model test
# input = '''분위기 좋은'''
# temp = output(input, new_model)
# labeled_temp = labeling(temp)
# print([targets[i] for i in range(len(labeled_temp)) if labeled_temp[i]==1])
# df_result = pd.DataFrame()
# df_result.index = targets
# df_result['점수'] = temp
# df_result

In [ ]:
class CombinedModel(Sequential):
    def __init__(self, s_bert, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.s_bert = s_bert
        self.model = model

    # def build(self, input_shape):
    #     super().build(input_shape)
    
    def predict(self, input):
        vector = self.s_bert.encode(input)
        vector = pd.DataFrame(vector).T
        return self.model.predict(vector)[0].tolist()

combined_model = CombinedModel(s_bert, model)

input = '넓고 매장이 깨끗한 맛집'
print(combined_model.predict(input))

1/1 [==============================] - 0s 37ms/step
[0.014227721840143204, 7.07280065626037e-08, 0.9574210047721863, 0.0007504008244723082, 0.0007229935145005584, 0.0036875009536743164, 0.0008101442363113165, 0.00017986638704314828, 0.03195584565401077, 0.8392485976219177, 0.00022622442338615656, 8.641440217616037e-05, 0.00017008197028189898, 0.0005744951195083559, 0.02659190446138382]


In [ ]:
# input_shape = (None, 768)
# _ = combined_model.build(input_shape)
# combined_model.save('combined_model')

In [ ]:
# temp_dir = 'C:\AI_bootcamp_python\CP2/bert_model'
# torch.save(s_bert.state_dict(), temp_dir + '/model.bin')

In [ ]:
# model.save('sequential_model')

In [ ]:
# very_new_model = keras.models.load_model('sequential_model')

In [ ]:
# new_combined_model = CombinedModel(s_bert, very_new_model)
# new_combined_model.predict('가성비')

In [ ]:
# s_bert_new = SentenceTransformer('C:\AI_bootcamp_python\CP2/bert_model/model.bin')
# s_bert_new.model = SentenceTransformer.SentenceTransformerModel.from_pretrained(temp_dir)
# s_bert_new.model.load_state_dict(torch.load(temp_dir + '/model.bin'))

In [ ]:
# # pickling the model
# import pickle

# with open('combined_model.pkl', 'wb') as file:
#     pickle.dump(combined_model, file)